<a href="https://colab.research.google.com/github/MapleWolfe/Milestone_2/blob/ning/CNN_shcool_assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms

In [ ]:
torch.manual_seed(0)
torch.set_num_threads(4)
torch.set_num_interop_threads(4)

In [ ]:
trainTransform = transforms.Compose([#add yours here!
                                     transforms.ToTensor(),
                                     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
                                    ])
testTransform = transforms.Compose([transforms.ToTensor(),
                                     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
                                    ])

In [ ]:
class CNNModel(nn.Module):
    def __init__(self, hiddenSize, outChannels, dropoutRate, activate):
        super().__init__()
        self.outChannels = outChannels
        self.activate = nn.Sigmoid() if activate == "Sigmoid" else nn.ReLU()
        self.conv1 = nn.Conv2d(3, 24, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(24, outChannels, 5)
        self.dense1 = nn.Linear(outChannels * 5 * 5, hiddenSize)
        self.dropout = nn.Dropout(dropoutRate)
        self.dense2 = nn.Linear(hiddenSize, 10)

    def forward(self, x):
        x = self.pool(self.activate(self.conv1(x)))
        x = self.pool(self.activate(self.conv2(x)))
        x = x.view(-1, self.outChannels * 5 * 5)
        x = self.dropout(self.activate(self.dense1(x)))
        return self.dense2(x)

In [ ]:
# Number of neurons in the first fully-connected layer
hiddenSize = 100
# Number of feature filters in second convolutional layer
numFilters = 25
# Dropout rate
dropoutRate = 0
# Activation function
activation = "ReLU"
# Learning rate
learningRate = 0.001
# Momentum for SGD optimizer
momentum = 0.9
# Number of training epochs
numEpochs = 10

In [ ]:
root_dir = 'assets_week2'
trainDataset = torchvision.datasets.CIFAR10(root=root_dir, train=True, download=True, transform=trainTransform)
trainLoader = torch.utils.data.DataLoader(trainDataset, batch_size=4, shuffle=True, num_workers=2)
testDataset = torchvision.datasets.CIFAR10(root=root_dir, train=False, download=True, transform=testTransform)
testLoader = torch.utils.data.DataLoader(testDataset, batch_size=4, shuffle=False, num_workers=2)

In [ ]:

cnn = CNNModel(hiddenSize, numFilters, dropoutRate, activation)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(list(ann.parameters()) + list(cnn.parameters()), lr=learningRate, momentum=momentum)

print('>>> Beginning training!')

cnn.train()
for epoch in range(numEpochs):  # loop over the dataset multiple times
    cnnRunningLoss = 0
    for i, (inputs, labels) in enumerate(trainLoader, 0):


        optimizer.zero_grad()

        # Forward propagation

        cnnOutputs = cnn(inputs)

        # Backpropagation

        cnnLoss = criterion(cnnOutputs, labels)

        cnnLoss.backward()

        # Gradient update
        optimizer.step()


        cnnRunningLoss += cnnLoss.item()
        if (i+1) % 2000 == 0:    # print every 2000 mini-batches
            print('Epoch [{}/{}], Step [{}/{}],  CNN Loss: {}'.format(epoch + 1, numEpochs, i + 1, len(trainDataset)//4,  cnnRunningLoss/2000))
            cnnRunningLoss = 0

print()
print('>>> Beginning validation!')

cnn.eval()
cnnCorrect =  0
total = 0
for inputs, labels in testLoader:


    cnnOutputs = cnn(inputs)

    _, cnnPredicted = torch.max(cnnOutputs.data, 1)
    total += labels.size(0)

    cnnCorrect += (cnnPredicted == labels).sum().item()
print('CNN validation accuracy: {}%'.format(cnnCorrect / total * 100))